# Distance Network 

In [5]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import pickle
from tqdm import tqdm

from geopy.distance import vincenty, great_circle

from scipy import stats
from IPython.display import display

In [2]:
with open(os.path.join("pickle", 'conflict.pickle'), 'rb') as data_source:
    conflict_df = pickle.load(data_source)

In [3]:
with open(os.path.join("pickle", 'refugee.pickle'), 'rb') as data_source:
    refugee_df = pickle.load(data_source)

## Create a network of distances

We split this network in two types of node: events and conflicts.
Each conflict location is the centroid of all its events (mean longitude and latitude).
Each event is linked to the conflict it belongs to by an edge whose weight represents the distance to the center of the conflict.
All the conflicts will be linked together by an edge describing the distance.

For each conflict node, we will also extract the statistics of its event distances using `stats.describe`.

This network will be used to find spatial relationship between various conflicts.
Also, because each node has a longitude and latitude information, it will be possible to print it on a map and
selectively print links between event and conflicts or between conflicts.

In [6]:
display(conflict_df.head(1))
display(refugee_df.head(1))

,id,year,type_of_violence,conflict_new_id,conflict_name,side_a_new_id,gwnoa,side_a,gwnob,side_b_new_id,...,country,country_id,region,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best
0,4,2010,1,230,Yemen (North Yemen):Government,123,678.0,Government of Yemen (North Yemen),NaN,881,...,Yemen (North Yemen),678,Middle East,2010,2010,2,0,0,0,2


,year,refugee,asylum,internally_displaced,stateless,others,total,country_id
0,1989.0,100786.0,0.0,0.0,0.0,0.0,100786.0,490.0


In [7]:
# graph indexed
conflict_df = conflict_df.set_index('id')
display(conflict_df.head(2))

,year,type_of_violence,conflict_new_id,conflict_name,side_a_new_id,gwnoa,side_a,gwnob,side_b_new_id,side_b,...,country,country_id,region,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best
id,,,,,,,,,,,,,,,,,,,,,
4,2010,1,230,Yemen (North Yemen):Government,123,678.0,Government of Yemen (North Yemen),NaN,881,AQAP,...,Yemen (North Yemen),678,Middle East,2010,2010,2,0,0,0,2
5,2011,3,715,Government of Yemen (North Yemen) - Civilians,123,678.0,Government of Yemen (North Yemen),NaN,1,Civilians,...,Yemen (North Yemen),678,Middle East,2011,2011,0,0,0,0,0


In [8]:
def distance_between_nodes(graph, node_1_id, node_2_id):
    """Get distance between two nodes by using their latitude and longitude property"""
    pos_1 = (graph.node[node_1_id]["latitude"], graph.node[node_1_id]["longitude"])
    pos_2 = (graph.node[node_2_id]["latitude"], graph.node[node_2_id]["longitude"])
    # If vincenty doesn't converge, put None as weight
    try:
        distance = vincenty(pos_1, pos_2).km
    except:
        # Try to get great circle distance instead
        try:
            distance = great_circle(pos_1, pos_2).km
        except:
            print("Error: failed to get distance between node {} and node {}".format(node_1_id, node_2_id))
            print("Node 1 positions: {}".format(pos_1))
            print("Node 2 positions: {}".format(pos_2))
            distance = None
    return distance

In [9]:
def get_conflict_lat_long(conflict_id):
    """Get the average latitude and longitude for a particular conflict id"""
    longitude = conflict_df[conflict_df.conflict_new_id == conflict_id].longitude.mean()
    latitude = conflict_df[conflict_df.conflict_new_id == conflict_id].latitude.mean()
    return (latitude, longitude)

In [10]:
def get_conflict_stats_by_year(conflict_id):
    """Get the conflict statistics by year"""
    local_conflict_df = conflict_df[conflict_df.conflict_new_id == conflict_id]
    local_conflict_dict = {}
    for year in local_conflict_df.year.unique():
        year_local_conflict_df = local_conflict_df[local_conflict_df.year == year]
        yearly_dict = {}
        yearly_dict["year"] = year
        yearly_dict["conflict_id"] = conflict_id 
        yearly_dict["type_of_violence"] = year_local_conflict_df.iloc[0].type_of_violence
        yearly_dict["side_a"] = year_local_conflict_df.iloc[0].side_a
        yearly_dict["side_b"] = year_local_conflict_df.iloc[0].side_b
        yearly_dict["gwnoa"] = year_local_conflict_df.iloc[0].gwnoa
        yearly_dict["gwnob"] = year_local_conflict_df.iloc[0].gwnob
        yearly_dict["deaths_a"] = year_local_conflict_df.deaths_a.sum()
        yearly_dict["deaths_b"] = year_local_conflict_df.deaths_b.sum()
        yearly_dict["deaths_civilians"] = year_local_conflict_df.deaths_civilians.sum()
        yearly_dict["best"] = year_local_conflict_df.best.sum() 
        yearly_dict["latitude"] = year_local_conflict_df.latitude.mean()
        yearly_dict["longitude"] = year_local_conflict_df.longitude.mean()
        local_conflict_dict[year] = yearly_dict
    return local_conflict_dict

After having defined some auxiliary functions, we can finally create the graph:

In [11]:
distance_graph = nx.Graph()

# Create a node for each conflict event
distance_graph.add_nodes_from(conflict_df.index.values, nature="event", year=conflict_df.year)

# Set longitude and latitude for each node
for index in conflict_df.index.values:
    distance_graph.node[index]["longitude"] = conflict_df.loc[index, "longitude"]
    distance_graph.node[index]["latitude"] = conflict_df.loc[index, "latitude"]

# Create a node for each unique conflict, use a special node id to avoid conflict with events
for conflict_id in tqdm(conflict_df.conflict_new_id.unique()):
    conflict_node_name = "conflict_{}".format(conflict_id)
    distance_graph.add_node(conflict_node_name, nature="conflict")
    
    # Get the average position for the conflict
    latitude, longitude = get_conflict_lat_long(conflict_id)
    distance_graph.node[conflict_node_name]["longitude"] = longitude
    distance_graph.node[conflict_node_name]["latitude"] = latitude
    
    # Set informations about the conflict
    distance_graph.node[conflict_node_name]["yearly_dict"] = get_conflict_stats_by_year(conflict_id)
    distance_graph.node[conflict_node_name]["conflict_id"] = conflict_id

100%|██████████| 997/997 [00:12<00:00, 79.97it/s]


In [12]:
# Create edges from event to their conflict
for conflict_id in tqdm(conflict_df.conflict_new_id.unique()):
    conflict_node_name = "conflict_{}".format(conflict_id)
    
    for event_id in conflict_df[conflict_df.conflict_new_id == conflict_id].index.values:
        distance = distance_between_nodes(distance_graph, conflict_node_name, event_id)
        distance_graph.add_edge(conflict_node_name, event_id, weight=distance, nature="event_to_conflict")


100%|██████████| 997/997 [00:05<00:00, 184.27it/s]


In [13]:
# Extract the distance between all conflicts 
for conflict_id_1 in tqdm(conflict_df.conflict_new_id.unique()):
    conflict_node_name_1 = "conflict_{}".format(conflict_id_1)
    
    for conflict_id_2 in conflict_df.conflict_new_id.unique():   
        conflict_node_name_2 = "conflict_{}".format(conflict_id_2)   
        
        # No self loop
        if conflict_id_1 == conflict_id_2:
            pass
        
        # Don't do two times the same edge
        if distance_graph.has_edge(conflict_node_name_1, conflict_node_name_2):
            pass
        
        distance_graph.add_edge(conflict_node_name_1, conflict_node_name_2, 
                  weight = distance_between_nodes(distance_graph, conflict_node_name_1, conflict_node_name_2),
                  nature = "conflict_to_conflict")
        

100%|██████████| 997/997 [00:41<00:00, 24.01it/s]


In [14]:
def get_event_weights(graph, node, nature="event_to_conflict"):
    """Return the event weights for the given nature. Can be an array of natures"""
    edges = [e[2] for e in distance_graph.edges_iter(nbunch=node, data=True)] 
    if type(nature) == str:
        return [d["weight"] for d in edges if d["nature"] == nature]
    else:
        weights = [[] for x in range(len(nature))]
        for index, item in enumerate(nature):
            weights[index] = [d["weight"] for d in edges if d["nature"] == item]
        return weights

In [15]:
# Extract conflict statistics
for node in tqdm(distance_graph.nodes()):
    if distance_graph.node[node]["nature"] == "event":
        continue
    
    weights = get_event_weights(distance_graph, node, nature=["event_to_conflict", "conflict_to_conflict"])
    
    distance_graph.node[node]["events_stats"] = stats.describe(weights[0])        

 96%|█████████▌| 130647/136178 [00:00<00:00, 1306004.18it/s]/home/vinz/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3126: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/vinz/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 136178/136178 [00:01<00:00, 109087.15it/s] 


In [16]:
with open(os.path.join("pickle", 'distance_nx.pickle'), 'wb') as out:
    pickle.dump(distance_graph, out)

In [17]:
with open(os.path.join("pickle", 'distance_nx.pickle'), 'rb') as data_source:
    distance_graph = pickle.load(data_source) 